In [6]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

# Define two LLMs
teacher_llm = ChatGroq(groq_api_key="", model_name="llama-3.3-70b-versatile")
student_llm = ChatGroq(groq_api_key="", model_name="Gemma2-9b-It")

# Define state
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# Teacher function
def teacher(state: State):
    teacher_prompt = f"You are a teacher. Guide the student and ask questions based on: {state['messages'][-1].content}"
    teacher_response = teacher_llm.invoke([HumanMessage(content=teacher_prompt)])
    return {"messages": [AIMessage(content=teacher_response.content)]}  # Use AIMessage for teacher

# Student function
def student(state: State):
    student_prompt = f"You are a student. Respond to the teacher and ask a follow-up question based on: {state['messages'][-1].content}"
    student_response = student_llm.invoke([HumanMessage(content=student_prompt)])
    return {"messages": [HumanMessage(content=student_response.content)]}  # Use HumanMessage for student

# Add nodes
graph_builder.add_node("teacher", teacher)
graph_builder.add_node("student", student)

# Define flow
graph_builder.add_edge(START, "teacher")
graph_builder.add_edge("teacher", "student")
graph_builder.add_edge("student", "teacher")  # Loop continues

graph = graph_builder.compile()

# Conversation loop with a recursion limit
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("Good Bye!")
        break  # Exits the while loop correctly

    state = {"messages": [HumanMessage(content=user_input)]}
    turn_count = 0

    for event in graph.stream(state):
        for value in event.values():
            message = value["messages"][0]
            if isinstance(message, AIMessage):
                print(f"Teacher: {message.content}")  # Print as Teacher
            else:
                print(f"Student: {message.content}")  # Print as Student

            turn_count += 1
            if turn_count >= 5:
                user_input = input("Continue conversation? (or type a new question): ")
                if user_input.lower() in ["quit", "q"]:
                    print("Good Bye!")
                    break  # Exits the inner loop

        if user_input.lower() in ["quit", "q"]:
            break  # Ensures we also exit the outer while loop

User: what is photosynthesis
Teacher: I'm excited to explore the topic of photosynthesis with you. Photosynthesis is a vital process that occurs in plants, algae, and some bacteria. It's the way they produce their own food using energy from the sun.

To start, can you tell me what you already know about photosynthesis? What comes to mind when you hear the word "photosynthesis"? 

(Also, don't worry if you're not sure or don't know much about it yet. We'll learn and discover together.)
Student: That sounds fascinating!  When I hear "photosynthesis," I think of plants using sunlight to grow.  I'm curious,  **what are the main ingredients that plants need besides sunlight to make their food?** 


Teacher: That's a great start. You're on the right track by thinking of plants using sunlight to grow. Now, let's dive deeper into the process of photosynthesis. 

You asked about the main ingredients that plants need besides sunlight to make their food. Can you take a guess? What do you think pl

In [7]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
import time

# Define your Groq API key
GROQ_API_KEY = ""

# Define two LLMs
teacher_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")
student_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="Gemma2-9b-It")

# Token limits based on Groq's rate limits
TOKEN_LIMITS = {
    "llama-3.3-70b-versatile": {"TPM": 6000, "TPD": 100000},
    "Gemma2-9b-It": {"TPM": 15000, "TPD": 500000}
}

# Token tracking variables
total_tokens_used = 0
tokens_used_last_minute = 0
start_time = time.time()

# Function to estimate token count
def count_tokens(text):
    return len(text.split())  # Approximate token count using word count

# Function to check token limits and warn
def check_token_limits(tokens_used, model_name):
    global total_tokens_used, tokens_used_last_minute
    total_tokens_used += tokens_used
    tokens_used_last_minute += tokens_used

    tpm_limit = TOKEN_LIMITS[model_name]["TPM"]
    tpd_limit = TOKEN_LIMITS[model_name]["TPD"]

    tokens_available_tpm = tpm_limit - tokens_used_last_minute
    tokens_available_tpd = tpd_limit - total_tokens_used

    # Reset per-minute counter every 60 seconds
    if time.time() - start_time >= 60:
        tokens_used_last_minute = 0

    # Display available tokens
    print(f"\n(Tokens Used: {tokens_used}, Total: {total_tokens_used}, Available (TPM: {tokens_available_tpm}, TPD: {tokens_available_tpd}))")

    # Warnings
    if tokens_used_last_minute >= 0.8 * tpm_limit:
        print("WARNING: You are close to your Tokens Per Minute (TPM) limit.")

    if total_tokens_used >= 0.8 * tpd_limit:
        print("WARNING: You are close to your Tokens Per Day (TPD) limit.")

# Teacher function
def teacher(state: State):
    teacher_prompt = f"You are a teacher. Guide the student and ask questions based on: {state['messages'][-1].content}"
    teacher_response = teacher_llm.invoke([HumanMessage(content=teacher_prompt)])

    tokens_used = count_tokens(teacher_response.content)
    check_token_limits(tokens_used, "llama-3.3-70b-versatile")

    return {"messages": [AIMessage(content=teacher_response.content)]}  # Use AIMessage for teacher

# Student function
def student(state: State):
    student_prompt = f"You are a student. Respond to the teacher and ask a follow-up question based on: {state['messages'][-1].content}"
    student_response = student_llm.invoke([HumanMessage(content=student_prompt)])

    tokens_used = count_tokens(student_response.content)
    check_token_limits(tokens_used, "Gemma2-9b-It")

    return {"messages": [HumanMessage(content=student_response.content)]}  # Use HumanMessage for student

# Define state
class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("teacher", teacher)
graph_builder.add_node("student", student)

# Define flow
graph_builder.add_edge(START, "teacher")
graph_builder.add_edge("teacher", "student")
graph_builder.add_edge("student", "teacher")  # Loop continues

graph = graph_builder.compile()

# Conversation loop with a recursion limit
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q"]:
        print("\nGoodbye!")
        break  # Exits the while loop correctly

    state = {"messages": [HumanMessage(content=user_input)]}
    turn_count = 0

    for event in graph.stream(state):
        for value in event.values():
            message = value["messages"][0]
            if isinstance(message, AIMessage):
                print(f"\nTeacher: {message.content}")  # Print as Teacher
            else:
                print(f"\nStudent: {message.content}")  # Print as Student

            turn_count += 1
            if turn_count >= 5:
                user_input = input("Continue conversation? (or type a new question): ")
                if user_input.lower() in ["quit", "q"]:
                    print("\nGoodbye!")
                    break  # Exits the inner loop

        if user_input.lower() in ["quit", "q"]:
            break  # Ensures we also exit the outer while loop

# Display final token stats after quitting
tokens_available_tpd = TOKEN_LIMITS["llama-3.3-70b-versatile"]["TPD"] - total_tokens_used
print("\nFinal Token Usage:")
print(f"   Total Tokens Used: {total_tokens_used}")
print(f"   Tokens Available (TPD): {tokens_available_tpd} (out of {TOKEN_LIMITS['llama-3.3-70b-versatile']['TPD']})")


User: what is photosynthesis

(Tokens Used: 46, Total: 46, Available (TPM: 5954, TPD: 99954))

Teacher: I'd love to explore the concept of photosynthesis with you. To start, can you tell me what you've heard about photosynthesis before? Maybe you've learned about it in school or heard about it from someone else. What comes to mind when you hear the word "photosynthesis"?

(Tokens Used: 48, Total: 94, Available (TPM: 14906, TPD: 499906))

Student: That sounds great! When I hear the word "photosynthesis," I think about plants using sunlight to make their own food.  I remember learning about chlorophyll and how it absorbs light energy, but I'm curious, what exactly is the food that plants make, and how do they use it? 





(Tokens Used: 127, Total: 221, Available (TPM: 5779, TPD: 99779))

Teacher: You're on the right track by thinking about plants using sunlight to make their own food through photosynthesis. That's a great foundation to build on.

To dive deeper, let's explore what you m

In [8]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict

# Define your Groq API key
GROQ_API_KEY = ""

# Define two LLMs
teacher_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")
student_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="Gemma2-9b-It")

# Token limits based on Groq's rate limits
TOKEN_LIMITS = {
    "llama-3.3-70b-versatile": {"TPM": 6000, "TPD": 100000},
    "Gemma2-9b-It": {"TPM": 15000, "TPD": 500000}
}

# Token tracking variables
total_tokens_used = 0

# Function to estimate token count
def count_tokens(text):
    return len(text.split())  # Approximate token count using word count

# Function to check token limits
def check_token_limits(tokens_used, model_name):
    global total_tokens_used
    total_tokens_used += tokens_used
    tokens_available_tpd = TOKEN_LIMITS[model_name]["TPD"] - total_tokens_used
    return tokens_available_tpd

# Teacher function
def teacher(state):
    last_message = state["messages"][-1].content

    # Check if the user wants to exit
    if "goodbye" in last_message.lower():
        return {"messages": [AIMessage(content="Goodbye! Have a great day.")], "end": True}

    teacher_prompt = f"You are a teacher. Guide the student and ask questions based on: {last_message}"
    teacher_response = teacher_llm.invoke([HumanMessage(content=teacher_prompt)])

    tokens_used = count_tokens(teacher_response.content)
    check_token_limits(tokens_used, "llama-3.3-70b-versatile")

    return {"messages": [AIMessage(content=teacher_response.content)]}

# Student function
def student(state):
    last_message = state["messages"][-1].content

    # Check if the user wants to exit
    if "goodbye" in last_message.lower():
        return {"messages": [HumanMessage(content="Goodbye! Thank you!")], "end": True}

    student_prompt = f"You are a student. Respond to the teacher and ask a follow-up question based on: {last_message}"
    student_response = student_llm.invoke([HumanMessage(content=student_prompt)])

    tokens_used = count_tokens(student_response.content)
    check_token_limits(tokens_used, "Gemma2-9b-It")

    return {"messages": [HumanMessage(content=student_response.content)]}

# Define state
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("teacher", teacher)
graph_builder.add_node("student", student)

graph_builder.add_edge(START, "teacher")
graph_builder.add_edge("teacher", "student")
graph_builder.add_edge("student", "teacher")  # Loop continues

graph = graph_builder.compile()

# Chat Interface Function
def chat_interface(user_input):
    messages = []
    state = {"messages": [HumanMessage(content=user_input)]}
    turn_count = 0

    for event in graph.stream(state, config={"recursion_limit": 10}):  # Set recursion limit
        for value in event.values():
            message = value["messages"][0]
            role = "teacher" if isinstance(message, AIMessage) else "student"

            # Append message with the correct alignment
            messages.append((role, message.content))

            turn_count += 1
            if turn_count >= 5 or value.get("end"):  # Stop condition
                return messages

    return messages

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("Teacher-Student Chatbot using langgraph")
    chatbot = gr.Chatbot(label="Teacher-Student Conversation", bubble_full_width=False)  # Make sure bubbles are not full width
    textbox = gr.Textbox(placeholder="Type your message...")
    submit = gr.Button("Send")

    def update_chat(user_input, chat_history):
        chat_responses = chat_interface(user_input)

        for role, message in chat_responses:
            if role == "teacher":
                chat_history.append((None, message))  # Teacher messages on the right
            else:
                chat_history.append((message, None))  # Student messages on the left

        return chat_history, ""

    submit.click(update_chat, [textbox, chatbot], [chatbot, textbox])

demo.launch()


<ipython-input-8-c3d165689c17>:106: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Teacher-Student Conversation", bubble_full_width=False)  # Make sure bubbles are not full width
<ipython-input-8-c3d165689c17>:106: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="Teacher-Student Conversation", bubble_full_width=False)  # Make sure bubbles are not full width


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91f905766a2f9df270.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
